In [ ]:
import pysal as ps
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline


In [ ]:
f = ps.open("data/usjoin.csv",'r')

To determine what is in the file, check the `header` attribute on the file object:

In [ ]:
f.header

Ok, lets pull in the name variable to see what we have

In [ ]:
name = f.by_col('Name')

In [ ]:
name

Now obtain per capital incomes in 1929 which is in the column associated with `1929`

In [ ]:
y1929 = f.by_col('1929')

In [ ]:
y1929

And now 2009

In [ ]:
y2009 = f.by_col("2009")

In [ ]:
y2009

These are read into regular Python lists which are not particularly well suited to efficient data analysis. So let's convert them to numpy arrays

In [ ]:
y2009 = np.array(y2009)

In [ ]:
y2009

Much better. But pulling these in and converting them a column at a time is tedious and error prone. So we will do all of this in a list comprehension.

In [ ]:
Y = np.array( [ f.by_col(str(year)) for year in range(1929,2010) ] ) * 1.0

In [ ]:
Y.shape

In [ ]:
Y = Y.transpose()

In [ ]:
Y.shape

In [ ]:
years = np.arange(1929,2010)

In [ ]:
plt.plot(years,Y[0])

In [ ]:
RY = Y / Y.mean(axis=0)

In [ ]:
plt.plot(years,RY[0])

In [ ]:
name = np.array(name)

In [ ]:
np.nonzero(name=='Ohio')

In [ ]:
plt.plot(years, RY[32], label='Ohio')
plt.plot(years, RY[0], label='Alabama')
plt.legend()

# Kernel Density (univariate, aspatial)

In [ ]:
from scipy.stats.kde import gaussian_kde

In [ ]:
density = gaussian_kde(Y[:,0])

In [ ]:
Y[:,0]

In [ ]:
density = gaussian_kde(Y[:,0])

In [ ]:
minY0 = Y[:,0].min()*.90
maxY0 = Y[:,0].max()*1.10
x = np.linspace(minY0, maxY0, 100)

In [ ]:
plt.plot(x,density(x))

In [ ]:
d2009 = gaussian_kde(Y[:,-1])

In [ ]:
minY0 = Y[:,-1].min()*.90
maxY0 = Y[:,-1].max()*1.10
x = np.linspace(minY0, maxY0, 100)

In [ ]:
plt.plot(x,d2009(x))

In [ ]:
minR0 = RY.min()

In [ ]:
maxR0 = RY.max()

In [ ]:
x = np.linspace(minR0, maxR0, 100)

In [ ]:
d1929 = gaussian_kde(RY[:,0])

In [ ]:
d2009 = gaussian_kde(RY[:,-1])

In [ ]:
plt.plot(x, d1929(x))
plt.plot(x, d2009(x))

In [ ]:
plt.plot(x, d1929(x), label='1929')
plt.plot(x, d2009(x), label='2009')
plt.legend()

## Markov Chains

In [ ]:
c = np.array([['b','a','c'],
['c','c','a'],
['c','b','c'],
['a','a','b'],
['a','b','c']])

In [ ]:
c

In [ ]:
m = ps.Markov(c)

In [ ]:
m.classes

In [ ]:
m.transitions

In [ ]:
m.p

### State Per Capita Incomes

In [ ]:
f = ps.open("data/usjoin.csv")
pci = np.array([f.by_col[str(y)] for y in range(1929,2010)])
pci.shape

Put series into cross-sectional quintiles (i.e., quintiles for each year)

In [ ]:
q5 = np.array([ps.Quantiles(y).yb for y in pci]).transpose()

In [ ]:
q5.shape

In [ ]:
q5[:,0]

In [ ]:
pci.shape

In [ ]:
pci[0]

we are looping over the rows of y which is ordered Txn (rows are cross sections, row 0 is the cross-section for period 0

In [ ]:
m5 = ps.Markov(q5)

In [ ]:
m5.classes

In [ ]:
m5.transitions

In [ ]:
m5.p

In [ ]:
m5.steady_state

In [ ]:
fmpt = ps.ergodic.fmpt(m5.p)

In [ ]:
fmpt

For a state with income in the first quintile, it takes on average 11.5 years for it to first enter the second quintile, 29.6 to get to the third quintile, 53.4 years to enter the fourth, and 103.6 years to reach the richest quintile.

## Spatial Markov

In [ ]:
pci.shape

In [ ]:
pci = pci.T

In [ ]:
pci.shape

In [ ]:
rpci = pci / pci.mean(axis=0)

In [ ]:
rpci[:,0]

In [ ]:
rpci[:,0].mean()

Read in a GAL file to construct our W

In [ ]:
w = ps.open(ps.examples.get_path("states48.gal")).read()

In [ ]:
w.n

In [ ]:
w.transform = 'R'

In [ ]:
sm = ps.Spatial_Markov(rpci, w, fixed=True, k=5)

In [ ]:
sm.p

In [ ]:
for p in sm.P:
    print p

In [ ]:
sm.S

In [ ]:
for f in sm.F:
    print f

## LISA Markov

In [ ]:
lm = ps.LISA_Markov(pci,w)

In [ ]:
lm.classes 

In [ ]:
lm.transitions

In [ ]:
lm.p

In [ ]:
lm.steady_state

In [ ]:
ps.ergodic.fmpt(lm.p)

### Test of independence of own chains and lag chains

In [ ]:
lm.transitions

In [ ]:
lm.expected_t

In [ ]:
lm.chi_2